# Looking at RECS

 Data are the responses from the 2009 Residential Energy Consumption Survey from the EIA: http://www.eia.gov/consumption/residential/
 
 Similar data is available for every four years since 1979

Goal: inform policy makers on where and what type of housing units that can most effectively boost residential energy efficiency in the US.

In [33]:
%matplotlib inline
import pandas as pd
import numpy as np
#import openpyxl

## See public_layout.csv file

In [26]:
pl = pd.DataFrame.from_csv('cap-data/public_layout.csv',index_col=None, 
                            encoding = 'utf-8')

In [27]:
pl.size

4655

## See recs2009_public.csv file

In [28]:
rp = pd.DataFrame.from_csv('cap-data/recs2009_public.csv', index_col = None, encoding = 'utf-8')

In [29]:
rp.head()

,DOEID,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,...,SCALEEL,KAVALNG,PERIODNG,SCALENG,PERIODLP,SCALELP,PERIODFO,SCALEFO,PERIODKR,SCALEKER
0,1,2,4,12,2,2471.679705,4742,1080,4953,1271,...,0,-2,-2,-2,-2,-2,-2,-2,-2,-2
1,2,4,10,26,2,8599.172010,2662,199,2688,143,...,0,1,1,0,-2,-2,-2,-2,-2,-2
2,3,1,1,1,5,8969.915921,6233,505,5741,829,...,0,3,5,3,-2,-2,-2,-2,-2,-2
3,4,2,3,7,2,18003.639600,6034,672,5781,868,...,3,3,5,3,-2,-2,-2,-2,-2,-2
4,5,1,1,1,3,5999.605242,5388,702,5313,797,...,0,1,1,0,-2,-2,-2,-2,-2,-2


## See recs2009_public_codebook.xlsx file

In [30]:
# Converted .xlsx to .csv, but is there a way to read in an excel file such that the 
# Response Codes and Labels and Code Descriptions come out cleaner and separated?
pc = pd.DataFrame.from_csv('cap-data/recs2009_public_codebook.csv', index_col = 0, encoding = 'utf-8')

In [31]:
pc.head(20)

,Variable Description,Response Codes and Labels,Code Desciption
Variable Name,,,
DOEID,Unique identifier for each respondent,00001 - 12083,Unique identifier for each respondent
REGIONC,Census Region,1\r2\r3\r4,Northeast Census Region\rMidwest Census Region...
DIVISION,Census Division,1\r2\r3\r4\r5\r6\r7\r8\r9\r10,"New England Census Division (CT, MA, ME, NH, R..."
REPORTABLE_DOMAIN,Reportable states and groups of states,1\r2\r3\r4\r5\r6\r7\r8\r9\r10\r11\r12\r13\r14\...,"\rConnecticut, Maine, New Hampshire, Rhode Isl..."
TYPEHUQ,Type of housing unit,1\r2\r3\r4\r5,Mobile Home\rSingle-Family Detached\rSingle-Fa...
NWEIGHT,Final sample weight,NaN,Final sample weight
HDD65,"Heating degree days in 2009, base temperature 65F",NaN,"Heating degree days in 2009, base temperature 65F"
CDD65,"Cooling degree days in 2009, base temperature 65F",NaN,"Cooling degree days in 2009, base temperature 65F"
HDD30YR,"Heating degree days, 30-year average 1981-2010...",NaN,"Heating degree days, 30-year average 1981-2010..."


In [32]:
pc.loc['DOEID',]

Variable Description         Unique identifier for each respondent
Response Codes and Labels                            00001 - 12083
Code Desciption              Unique identifier for each respondent
Name: DOEID, dtype: object

In [34]:
np.arange(1979,2009,4)

array([1979, 1983, 1987, 1991, 1995, 1999, 2003, 2007])

# Looking at Largest Dataset for Global Energy Forecasting Competition

In [35]:
test = pd.DataFrame.from_csv('cap-data/Load_history.csv',index_col=None, 
                            encoding = 'utf-8')

In [36]:
test.head(30)

,zone_id,year,month,day,h1,h2,h3,h4,h5,h6,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
0,1,2004,1,1,"16,853","16,450","16,517","16,873","17,064","17,727",...,"13,518","13,138","14,130","16,809","18,150","18,235","17,925","16,904","16,162","14,750"
1,1,2004,1,2,"14,155","14,038","14,019","14,489","14,920","16,072",...,"16,127","15,448","15,839","17,727","18,895","18,650","18,443","17,580","16,467","15,258"
2,1,2004,1,3,"14,439","14,272","14,109","14,081","14,775","15,491",...,"13,507","13,414","13,826","15,825","16,996","16,394","15,406","14,278","13,315","12,424"
3,1,2004,1,4,"11,273","10,415","9,943","9,859","9,881","10,248",...,"14,207","13,614","14,162","16,237","17,430","17,218","16,633","15,238","13,580","11,727"
4,1,2004,1,5,"10,750","10,321","10,107","10,065","10,419","12,101",...,"13,845","14,350","15,501","17,307","18,786","19,089","19,192","18,416","17,006","16,018"
5,1,2004,1,6,"15,742","15,682","16,132","16,761","17,909","20,234",...,"18,762","19,162","21,509","25,314","28,060","28,768","28,919","28,653","27,406","26,507"
6,1,2004,1,7,"26,014","26,447","27,286","27,923","29,130","31,503",...,"20,617","21,013","23,676","27,329","29,685","29,838","29,806","28,704","27,069","25,708"
7,1,2004,1,8,"25,104","25,122","25,464","25,715","26,219","28,552",...,"22,421","22,883","24,436","26,555","27,394","27,486","26,890","25,529","23,869","22,278"
8,1,2004,1,9,"21,175","21,056","21,241","22,062","23,026","25,610",...,"24,955","24,932","25,497","27,668","28,784","28,113","27,311","26,327","24,967","23,824"
9,1,2004,1,10,"23,405","23,507","24,067","24,786","25,418","26,631",...,"22,257","22,457","23,909","27,515","29,526","30,073","30,858","30,698","30,208","30,056"
